In [ ]:
# importing libraries

import open3d, os
import ctypes
from ctypes import *
from glob import glob

# Data augmentation

In [ ]:
from data_augmentation import rotate_scale

# read all the files in the folder dataset/original_dataset
files = glob("./new_dataset/train/input/quads/*")

for file in files:
    rotate_scale(file)


In [ ]:
from data_augmentation import data_preprocessing

# read all the files in the folder dataset/original_dataset
files = glob("./new_dataset/train/input/quads/*")

for file in files:
    data_preprocessing(file)


In [ ]:
from utils import generate_output
import numpy as np

files_quads = glob("./new_dataset/train/input/quads/*")

for file in files_quads:
  file_name = os.path.basename(file)
  file_triangle = "./new_dataset/train/input/triangles/" + file_name

  orientation_fields = generate_output(file_triangle, file)
  orientation_fields_reshaped = orientation_fields.reshape(orientation_fields.shape[0], -1)


  # save the orientation fields in a txt file
  output_file = "./new_dataset/train/output/" + file_name
  np.savetxt(output_file, orientation_fields_reshaped)


  print("Orientation fields saved in: ", output_file)

## Loading mesh to retopologize

In [ ]:
# loading the shared library with C++
shared_library_path = glob(os.path.join("./build", "**/*.so"), recursive=True)
assert shared_library_path, "No shared library found, please build the shared library first (README.md for instructions)"
shared_library = ctypes.CDLL(shared_library_path[0])

file_path = "./new_dataset/train/input/triangles"
file_output_path = os.path.join(file_path, "..", "..", "output")

files_triangles = glob("./new_dataset/train/input/triangles/*")

for file_triangle in files_triangles:

    mesh = open3d.io.read_triangle_mesh(file_triangle)

    vertices = mesh.vertices
    triangles = mesh.triangles

    num_vertices = len(vertices)
    num_triangles = len(triangles)

    # create file input.txt
    file = open("input.txt", "w")

    # write vertices
    for i in range(len(vertices)):
        file.write(str(vertices[i][0]) + " " + str(vertices[i][1]) + " " + str(vertices[i][2]) + "\n")

    # write triangles
    for i in range(len(triangles)):
        file.write(str(triangles[i][0]) + " " + str(triangles[i][1]) + " " + str(triangles[i][2]) + "\n")


    file.close()

    k = 3 # neighborhood size

    file_output = os.path.join(file_output_path, os.path.basename(file_triangle))

    # get the complete url of the file
    file_output = os.path.abspath(file_output)

    # modify the basename concatenating _principal_directions
    file_output = file_output.replace(".obj", "_principal_directions.txt")

    output = create_string_buffer(file_output.encode('utf-8'),size=100)
    shared_library.crest_lines(num_vertices, num_triangles, k, output)